In [1]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque
import numpy as np


In [2]:

def generate_data_block(num_samples, num_features):
    """
    Tạo một khối dữ liệu với num_samples mẫu và num_features đặc trưng.
    Mỗi mẫu bao gồm đặc trưng và nhãn.
    """
    # Tạo dữ liệu đặc trưng ngẫu nhiên
    features = np.random.rand(num_samples, num_features)
    
    # Tạo nhãn ngẫu nhiên (giả sử có 2 lớp: 0 và 1)
    labels = np.random.randint(0, 2, size=num_samples)
    
    # Kết hợp các đặc trưng và nhãn
    block = list(zip(features, labels))
    return block

def generate_data_stream(num_blocks, num_samples_per_block, num_features):
    """
    Tạo luồng dữ liệu gồm nhiều khối, mỗi khối chứa nhiều mẫu.
    """
    data_stream = []
    for i in range(num_blocks):
        block = generate_data_block(num_samples_per_block, num_features)
        data_stream.append(block)
    return data_stream

# Ví dụ tạo luồng dữ liệu S = (B1, B2, ..., Bt, Bt+1, ...)
num_blocks = 5  # Số lượng khối dữ liệu
num_samples_per_block = 10  # Số mẫu trong mỗi khối
num_features = 4  # Số lượng đặc trưng mỗi mẫu

# Tạo luồng dữ liệu
S = generate_data_stream(num_blocks, num_samples_per_block, num_features)


Block B1:
Features: [0.74322784 0.4244388  0.16541554 0.5508704 ], Label: 1
Features: [0.28997349 0.91004083 0.36895157 0.35040132], Label: 0
Features: [0.13906544 0.61485106 0.52777782 0.51462529], Label: 1
Features: [0.47004314 0.85237669 0.51939802 0.52876554], Label: 0
Features: [0.40860213 0.61952054 0.46528395 0.66570158], Label: 1
Features: [0.52577664 0.65459146 0.82538843 0.93587525], Label: 1
Features: [0.30464009 0.66746154 0.58490211 0.75420597], Label: 1
Features: [0.92788577 0.82062913 0.73585686 0.22956815], Label: 0
Features: [0.90845983 0.54089028 0.01487966 0.22479079], Label: 1
Features: [0.88070649 0.94244355 0.58452379 0.81115242], Label: 0


Block B2:
Features: [0.3171594  0.26085513 0.59585177 0.48676959], Label: 0
Features: [0.5251103  0.83156345 0.61998097 0.7549275 ], Label: 0
Features: [0.71808322 0.83509815 0.32435128 0.44041923], Label: 0
Features: [0.4861026  0.3947851  0.50386276 0.5058283 ], Label: 1
Features: [0.71884938 0.04351453 0.85634474 0.33140854

In [3]:
def train_random_forest_classifier(X, y, n_estimators=10):
    """
    Huấn luyện một bộ phân loại Random Forest trên dữ liệu X, y.
    n_estimators: Số lượng cây trong rừng ngẫu nhiên.
    """
    rf = RandomForestClassifier(n_estimators=n_estimators)
    rf.fit(X, y)
    return rf


def adaptive_ensemble_size(C, sample, epsilon=0.01, min_classifiers=3):
    return

def tendency_prediction(selected_classifiers, epsilon=0.01):
    return

def process_data_stream(S, m, k, epsilon=0.01):
    """
    Xử lý luồng dữ liệu S theo thuật toán với bộ phân loại Random Forest yếu.
    - S: Luồng dữ liệu (một danh sách các khối Bi)
    - m: Kích thước tối đa của tập hợp bộ phân loại C
    - k: Kích thước tối thiểu của C để tiến hành dự đoán
    """
    C = deque(maxlen=m)  # Sử dụng deque để lưu trữ C với kích thước tối đa m
    Pre = []
    for Bi in S:
        # Tách dữ liệu và nhãn từ khối dữ liệu Bi
        X = np.array([sample[0] for sample in Bi])  # Đặc trưng
        y = np.array([sample[1] for sample in Bi])  # Nhãn

        # Bước 2: Huấn luyện bộ phân loại yếu mới Ci từ khối Bi
        Ci = train_random_forest_classifier(X, y)

        # Bước 3-4: Thêm bộ phân loại vào tập hợp C
        if len(C) < m:
            C.append(Ci)
        else:
            C.append(Ci)  # Tự động loại bỏ bộ phân loại yếu cũ nhất nếu đã đạt kích thước tối đa

        # Bước 9-10: Nếu tập hợp C có kích thước nhỏ hơn k, tiếp tục chờ khối dữ liệu mới
        if len(C) < k:
            print("Kích thước C nhỏ hơn k, chờ khối dữ liệu mới...")
            continue
        
        # Tại đây, bạn có thể thêm logic để sử dụng tập hợp C để thực hiện dự đoán
        # Ví dụ: sử dụng các bộ phân loại trong C để dự đoán trên một khối dữ liệu mới

        print(f"Đã thêm bộ phân loại mới, kích thước C hiện tại: {len(C)}")
        print(f"Dự đoán cho khối Bi+1:")

        for sample, true_label in Bi:  # Dự đoán từng mẫu trong Bi+1
            # Bước 13: Dự đoán mẫu Bi+1 bằng AL2 và AL3
            selected_classifiers = adaptive_ensemble_size(C, sample, epsilon)
            predicted_class = tendency_prediction(selected_classifiers, epsilon)
            Pre[i].append(predicted_class)
    return C, Pre

m = 15
k = 3

process_data_stream(S, m, k)

Kích thước C nhỏ hơn k, chờ khối dữ liệu mới...
Kích thước C nhỏ hơn k, chờ khối dữ liệu mới...
Đã thêm bộ phân loại mới, kích thước C hiện tại: 3
Đã thêm bộ phân loại mới, kích thước C hiện tại: 4
Đã thêm bộ phân loại mới, kích thước C hiện tại: 5
